# TP1 - Aprendizaje Por Refuerzos
## Integrantes:
- **Azul Noguera** 
- **Paula Jordan** 
- **Rocio Gonzalez**
- **Valentina Pancaldi**

## Enunciado

Implementar un agente de aprendizaje por refuerzo y un ambiente en el cual el agente puede entrenarse. Esto incluye definir la noción de estado del ambiente, que puede no coincidir con el estado del juego.

## Preparación del Entorno

In [1]:
# importamos las clases
from utils import *
from jugador import * # importamos la clase Jugador
from diezmil import * # importamos la clase DiezMil

# importamos funciones de utilidad
from utils import puntaje_y_no_usados, separar, JUGADA_PLANTARSE, JUGADA_TIRAR, JUGADAS_STR

# importamos las librerias necesarias
import numpy as np
from collections import defaultdict
from tqdm import tqdm
import random
from random import randint
import csv

## Estado

In [2]:
class EstadoDiezMil:
    def __init__(self):
        """Definir qué hace a un estado de diez mil.
        Recordar que la complejidad del estado repercute en la complejidad de la tabla del agente de q-learning.
        """
        self.dados = []  # Lista de los valores actuales de los dados
        self.puntaje_actual = 0  # Puntaje acumulado en el turno actual
        self.turno_terminado = False  # Flag para indicar si el turno ha terminado

    def reset(self):
        """Reinicia el estado del juego a sus valores iniciales."""
        self.dados = self.lanzar_dados(6)  # Reiniciar los dados
        self.puntaje_actual = 0  # Reiniciar el puntaje acumulado
        self.turno_terminado = False  # Reiniciar el flag de turno terminado

    def lanzar_dados(self, cantidad: int) -> list[int]:
        """Simula el lanzamiento de una cantidad específica de dados.
        Args:
            cantidad (int): La cantidad de dados a lanzar.
        Returns:
            list[int]: Los valores obtenidos en los dados lanzados.
        """
        return [randint(1, 6) for _ in range(cantidad)]

    def obtener_estado(self):
        """Devuelve el estado actual del juego en un formato que el agente pueda utilizar."""
        return (self.dados, self.puntaje_actual, self.turno_terminado)

    def actualizar_estado(self, nuevos_dados: list[int], puntaje: int) -> None:
        """Modifica las variables internas del estado luego de una tirada.
        Args:
            nuevos_dados (list[int]): Dados no usados tras la tirada.
            puntaje (int): Puntaje obtenido en la tirada.
        """
    
        self.dados = nuevos_dados
        self.puntaje_actual += puntaje
        if len(self.dados) == 0:
            # Si no quedan dados no usados, el turno debe terminar
            self.turno_terminado = True
    
    def fin_turno(self):
        """Modifica el estado al terminar el turno.
        """
        self.turno_terminado = True  # Marca el turno como terminado

    def __str__(self):
        """Representación en texto de EstadoDiezMil.
        Ayuda a tener una versión legible del objeto.

        Returns:
            str: Representación en texto de EstadoDiezMil.
        """
        estado_str = f"Dados: {self.dados}, Puntaje Actual: {self.puntaje_actual}, Turno Terminado: {self.turno_terminado}"
        return estado_str   

## Ambiente

In [3]:
class AmbienteDiezMil:
    
    def __init__(self):
        """Definir las variables de instancia de un ambiente.
        ¿Qué es propio de un ambiente de 10.000?
        """
        self.estado = EstadoDiezMil()
        self.turno_terminado = False  # flag que indica si el turno terminó
        self.puntaje_total = 0  # Mantener un registro del puntaje total
        self.pasos_totales = 0  # Contador de pasos
        self.episodios_totales = 0  # Contador de episodios

    def reset(self):
        """
        Reinicia el estado del ambiente.
        """
        if self.pasos_totales > 0:  # Solo sumar episodios si se han realizado pasos
            self.episodios_totales += 1
        self.estado.reset()
        self.turno_terminado = False
        self.pasos_totales = 0
        
        # Aquí no reseteamos `self.puntaje_total` para mantener el acumulado entre episodios.
        return self.estado.obtener_estado()
    
    def step(self, accion):
        """
        Dada una acción devuelve una recompensa.
        El estado es modificado acorde a la acción y su interacción con el ambiente.
        Podría ser útil devolver si terminó o no el turno.
        
        Args:
            accion: Acción elegida por un agente.
        
        Returns:
            tuple[int, bool]: Una recompensa y un flag que indica si terminó el turno. 
        """
        # print(f"\n{'='*50}")
        # print(f"Acción Elegida: {accion.upper()}")

        # print(f"Estado Antes de la Acción: {self.estado}")
        
        self.pasos_totales += 1  # Incrementar el contador de pasos en cada acción
        # print(f"Paso {self.pasos_totales}: Acción = {accion}, Puntaje Total = {self.puntaje_total}")
        
        if accion == 'plantarse':
            self.turno_terminado = True
            recompensa = self.estado.puntaje_actual
            self.puntaje_total += recompensa  # Acumular la recompensa al puntaje total
            
        elif accion == 'tirar':
            nuevos_dados = self.estado.lanzar_dados(len(self.estado.dados))
            puntaje, dados_no_usados = puntaje_y_no_usados(nuevos_dados)
            # print(f"Tirada de Dados: {nuevos_dados}")
            # print(f"Puntaje Calculado: {puntaje}, Dados No Usados: {dados_no_usados}")
            self.estado.actualizar_estado(dados_no_usados, puntaje)
            recompensa = self.estado.puntaje_actual
            self.puntaje_total += recompensa  # Acumular la recompensa al puntaje total
            if recompensa == 0:
                self.turno_terminado = True
        else:
            raise ValueError("Acción no válida.")
        
        # print(f"Estado Después de la Acción: {self.estado}")
        # print(f"Recompensa Obtenida: {recompensa}")
        # print(f"Puntaje Total Acumulado: {self.puntaje_total}")
        # print(f"{'='*50}\n")
        
        return self.estado.obtener_estado(), recompensa, self.turno_terminado

## Agente

In [49]:
class AgenteQLearning:
    def __init__(self, ambiente: AmbienteDiezMil, alpha: float, gamma: float, epsilon: float):
        self.ambiente = ambiente
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.q_table = {}

    def elegir_accion(self, estado):
        # Convertir cada parte del estado a una tupla si es necesario
        estado = tuple(tuple(part) if isinstance(part, list) else part for part in estado)
        
        if random.uniform(0, 1) < self.epsilon:
            return random.choice(['plantarse', 'tirar'])
        else:
            if estado in self.q_table:
                return max(self.q_table[estado], key=self.q_table[estado].get)
            else:
                return random.choice(['plantarse', 'tirar'])

    def entrenar(self, episodios: int, verbose: bool = False) -> None:
        for episodio in tqdm(range(episodios)):
            estado = self.ambiente.reset()
            estado = tuple(tuple(part) if isinstance(part, list) else part for part in estado)
            terminado = False

            while not terminado:
                accion = self.elegir_accion(estado)
                nuevo_estado, recompensa, terminado = self.ambiente.step(accion)
                nuevo_estado = tuple(tuple(part) if isinstance(part, list) else part for part in nuevo_estado)

                if estado not in self.q_table:
                    self.q_table[estado] = {'plantarse': 0, 'tirar': 0}
                
                if nuevo_estado not in self.q_table:
                    self.q_table[nuevo_estado] = {'plantarse': 0, 'tirar': 0}

                mejor_q_nuevo_estado = max(self.q_table[nuevo_estado].values())
                self.q_table[estado][accion] += self.alpha * (
                    recompensa + self.gamma * mejor_q_nuevo_estado - self.q_table[estado][accion]
                )

                estado = nuevo_estado

                if verbose:
                    print(f"Paso: {episodio + 1}, Estado: {estado}, Acción: {accion}, Recompensa: {recompensa}")

            # # Verificar la condición de finalización del juego
            if self.ambiente.puntaje_total >= 10000:
                print(f"Entrenamiento terminado en el episodio {episodio + 1} con un puntaje total de {self.ambiente.puntaje_total}.")
                break

    def guardar_politica(self, filename: str):
        # Cargar los estados existentes en el archivo
        estados_existentes = set()
        try:
            with open(filename, 'r', newline='') as file:
                reader = csv.reader(file)
                next(reader)  # Saltar el encabezado
                for row in reader:
                    estado_str = row[0]  # El estado está en la primera columna
                    estados_existentes.add(estado_str)
        except FileNotFoundError:
            # Si el archivo no existe, continuar sin cargar estados
            pass

        # Abrir el archivo en modo append para agregar nuevas entradas
        with open(filename, 'a', newline='') as file:
            writer = csv.writer(file)

            # Si el archivo está vacío, escribir el encabezado
            if not estados_existentes:
                writer.writerow(['estado', 'plantarse', 'tirar'])

            # Guardar solo los estados nuevos (que no estén ya en el archivo)
            for estado, acciones in self.q_table.items():
                dados_ordenados = tuple(sorted(estado[0]))  # Suponiendo que los dados son el primer elemento del estado
                estado_ordenado = (dados_ordenados,) + estado[1:]  # Recombina el estado con los dados ordenados
                estado_str = str(estado_ordenado)  # Convertir el estado a una cadena para almacenarlo en el CSV
                
                if estado_str not in estados_existentes:
                    writer.writerow([estado_str, acciones['plantarse'], acciones['tirar']])

### Entrenamiento

In [5]:
class JugadorEntrenado(Jugador):
    def __init__(self, nombre: str, filename_politica: str):
        self.nombre = nombre
        self.politica = self._leer_politica(filename_politica)
        
    def _leer_politica(self, filename:str, SEP:str=','):
        """Carga una politica entrenada con un agente de RL, que está guardada
        en el archivo filename en formato CSV.

        Args:
            filename (str): Nombre/Path del archivo que contiene a una política almacenada. 
        """
        politica = {}
        with open(filename, 'r') as file:
            reader = csv.DictReader(file)
            for row in reader:
                estado = eval(row['estado'])  
                politica[estado] = {
                    'plantarse': float(row['plantarse']),
                    'tirar': float(row['tirar'])
                }
        return politica
    
    def jugar(self, puntaje_total:int, puntaje_turno:int, dados:list[int]) -> tuple[int,list[int]]:
        """
        Devuelve una jugada y los dados a tirar.
        Args:
            puntaje_total (int): Puntaje total del jugador en la partida.
            puntaje_turno (int): Puntaje en el turno del jugador
            dados (list[int]): Tirada del turno.
        Returns:
            tuple[int,list[int]]: Una jugada y la lista de dados a tirar.
        """

        # Calcular el puntaje actual y los dados no usados
        puntaje, no_usados = puntaje_y_no_usados(dados)

        # Definir el estado como una tupla con los componentes relevantes
        estado = (tuple(dados), puntaje_turno, False)  # Matching the structure used in training
        
        # Consultar la política para determinar la jugada
        jugada = self.politica.get(estado, None) 
        if jugada is not None:
            accion = max(jugada, key=jugada.get)  # Choose the action with the highest value
        else:
            print(f"Estado {estado} no encontrado en la política, usando acción por defecto.")
            accion = 'plantarse'  # Default action if the state is not found

        if accion == 'plantarse':
            return ('plantarse', [])
        elif accion == 'tirar':
            return ('tirar', no_usados)

### Ejecución

In [6]:
ambiente = AmbienteDiezMil()
agente = AgenteQLearning(ambiente, alpha=0.1, gamma=0.9, epsilon=0.2)
agente.entrenar(episodios=100, verbose=True)

 17%|█▋        | 17/100 [00:00<00:00, 22600.05it/s]

Paso: 1, Estado: ((3, 5, 5, 1, 5, 1), 0, False), Acción: plantarse, Recompensa: 0
Paso: 2, Estado: ((3, 4, 4, 3, 6, 3), 0, False), Acción: plantarse, Recompensa: 0
Paso: 3, Estado: ((4,), 750, False), Acción: tirar, Recompensa: 750
Paso: 3, Estado: ((4,), 750, False), Acción: plantarse, Recompensa: 750
Paso: 4, Estado: ((6, 5, 4, 1, 2, 2), 0, False), Acción: plantarse, Recompensa: 0
Paso: 5, Estado: ((4, 5, 3, 2, 6, 5), 0, False), Acción: plantarse, Recompensa: 0
Paso: 6, Estado: ((4, 2, 2, 3, 2, 6), 0, False), Acción: plantarse, Recompensa: 0
Paso: 7, Estado: ((2, 3, 6), 200, False), Acción: tirar, Recompensa: 200
Paso: 7, Estado: ((4,), 300, False), Acción: tirar, Recompensa: 300
Paso: 7, Estado: ((4,), 300, False), Acción: plantarse, Recompensa: 300
Paso: 8, Estado: ((3, 4), 300, False), Acción: tirar, Recompensa: 300
Paso: 8, Estado: ((3, 4), 300, False), Acción: plantarse, Recompensa: 300
Paso: 9, Estado: ((5, 1, 4, 1, 5, 6), 0, False), Acción: plantarse, Recompensa: 0
Paso: 10, E

### Guardado de Política

In [7]:
agente.guardar_politica("politica_qlearning.csv")

### Evaluación

In [28]:
jugador = JugadorEntrenado("Agente Trained", "politica_qlearning.csv")
print(jugador.politica)
resultado = jugador.jugar(puntaje_total=0, puntaje_turno=0, dados=[1, 1, 2, 4, 5, 6])

print(resultado)

{((1, 3, 4, 5, 6, 6), 0, False): {'plantarse': 0.0, 'tirar': 0.0}, ((1, 3, 5, 5, 6, 6), 0, False): {'plantarse': 0.0, 'tirar': 15.0}, ((2, 4, 6, 6), 150, False): {'plantarse': 15.0, 'tirar': 0.0}, ((1, 3, 3, 4, 4, 6), 0, False): {'plantarse': 0.0, 'tirar': 0.0}, ((1, 2, 3, 4, 5, 5), 0, False): {'plantarse': 0.0, 'tirar': 0.0}, ((1, 2, 4, 5, 6, 6), 0, False): {'plantarse': 0.0, 'tirar': 20.0}, ((3, 3, 4), 200, False): {'plantarse': 20.0, 'tirar': 0.0}, ((3, 3, 3, 5, 5, 5), 0, False): {'plantarse': 0.0, 'tirar': 0.0}, ((1, 1, 2, 4, 4, 5), 0, False): {'plantarse': 0.0, 'tirar': 0.0}, ((1, 2, 3, 3, 3, 5), 0, False): {'plantarse': 0.0, 'tirar': 0.0}, ((1, 2, 3, 3, 4, 5), 0, False): {'plantarse': 0.0, 'tirar': 0.0}, ((1, 1, 1, 2, 4, 5), 0, False): {'plantarse': 0.0, 'tirar': 25.0}, ((4, 6, 6), 250, False): {'plantarse': 25.0, 'tirar': 0.0}, ((2, 2, 2, 3, 4, 5), 0, False): {'plantarse': 0.0, 'tirar': 0.0}, ((1, 1, 4, 5, 6, 6), 0, False): {'plantarse': 0.0, 'tirar': 0.0}, ((2, 3, 4, 5, 6, 6), 

### Calculo promedio de varias partidas

Hiperparametros:
- alpha=0.1
- gamma=0.9
- epsilon=0.2

In [52]:
import numpy as np
from tqdm import tqdm

# Definir el número de partidas
numero_partidas = 1000 

# Almacenar los puntajes y el número de pasos de cada partida
puntajes_finales = []
episodios_por_partida = []

for _ in tqdm(range(numero_partidas)):
    # Crear una nueva instancia del ambiente y del agente para cada partida
    ambiente = AmbienteDiezMil()
    agente = AgenteQLearning(ambiente, alpha=0.1, gamma=0.9, epsilon=0.3)
    
    # Entrenar el agente en una partida de 100 episodios
    agente.entrenar(episodios=100, verbose=False)
    
    # Guardar el puntaje final de la partida y el número de pasos
    puntajes_finales.append(ambiente.puntaje_total)
    episodios_por_partida.append(ambiente.episodios_totales)  # Asumiendo que `ambiente.pasos_totales` guarda los pasos realizados

# Guardar la política después de todas las partidas
agente.guardar_politica("politica_qlearning.csv")

# Calcular el promedio de puntajes y el promedio de pasos
promedio_puntajes = np.mean(puntajes_finales)
promedio_pasos = np.mean(episodios_por_partida)

print(f"Promedio de puntajes después de {numero_partidas} partidas: {promedio_puntajes}")
print(f"Promedio de pasos por partida después de {numero_partidas} partidas: {promedio_pasos}")

 34%|███▍      | 34/100 [00:00<00:00, 26710.31it/s]


Entrenamiento terminado en el episodio 35 con un puntaje total de 13350.


 28%|██▊       | 28/100 [00:00<00:00, 42566.33it/s]


Entrenamiento terminado en el episodio 29 con un puntaje total de 10800.


 20%|██        | 20/100 [00:00<00:00, 40505.11it/s]


...

 25%|██▌       | 25/100 [00:00<00:00, 14979.66it/s]


 19%|█▉        | 19/100 [00:00<00:00, 23542.62it/s]


Entrenamiento terminado en el episodio 20 con un puntaje total de 11700.


100%|██████████| 1000/1000 [00:06<00:00, 153.34it/s]

Entrenamiento terminado en el episodio 54 con un puntaje total de 10550.
Promedio de puntajes después de 1000 partidas: 12345.9
Promedio de pasos por partida después de 1000 partidas: 21.696


In [51]:
print(f"Promedio de puntajes después de {numero_partidas} partidas: {promedio_puntajes}")
print(f"Promedio de pasos por partida después de {numero_partidas} partidas: {promedio_pasos}")

Promedio de puntajes después de 1000 partidas: 11914.35
Promedio de pasos por partida después de 1000 partidas: 19.718


In [48]:
print(puntajes_finales)
print(episodios_por_partida)

[10500, 10750, 13850, 10300, 11700, 16400, 10050, 11600, 14100, 10350, 10600, 10450, 10750, 10150, 11250, 10100, 12500, 11100, 10000, 18150, 10350, 10150, 10000, 12550, 10050, 18250, 12450, 19750, 10300, 11600, 13400, 10750, 10650, 25350, 10550, 10300, 10900, 12350, 14550, 10350, 11400, 12400, 10950, 10400, 11650, 10950, 10000, 20500, 10150, 12550, 11800, 12900, 10750, 12300, 11150, 10850, 11700, 12300, 12450, 10150, 10200, 10000, 10700, 16700, 10200, 11900, 10750, 10000, 10600, 10300, 10050, 12100, 10200, 11900, 11950, 10050, 12000, 11800, 13000, 10000, 10100, 11350, 17900, 10200, 10550, 11900, 10350, 10000, 12100, 10000, 11400, 11950, 10350, 11050, 10400, 11400, 11900, 16600, 10000, 11950, 10000, 10600, 10050, 14550, 20200, 10300, 11350, 10500, 11100, 11600, 12150, 10950, 10100, 10050, 10200, 11150, 13850, 10950, 10000, 12000, 12050, 10900, 10700, 10150, 11500, 10500, 10800, 10100, 12300, 10700, 11900, 10000, 10300, 11350, 11050, 11100, 10100, 10700, 11100, 10050, 10050, 10500, 11200